# Binary Bayes Net Inference

This is a quick notebook exercise to exemplify Bayes Net (BN) inference. 

Consider the following BN:  

![Imaginary SuperBowl Bayes Net Diagram](BN-NFL.png "Imaginary SuperBowl Bayes Net Diagram")


----
We can use the `BayesianNetwork` module from [pgmpy](https://pgmpy.org/) to construct this network:

In [6]:
import numpy as np
import pandas as pd

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/mmm/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/mmm/Library/Python/3.12/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/mmm/Library/Python/3.12/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File 

In [7]:
# Define Bayesian Network structure
model = BayesianNetwork([('F', 'Q'), ('F', 'D'), ('Q', 'W'), ('D', 'W')])

# Define CPDs
cpd_f = TabularCPD(variable='F', variable_card=2, values=[[0.3], [0.7]], state_names={"F":["low", "high"]})
cpd_q = TabularCPD(variable='Q', variable_card=2, values=[[0.5, 0.2], [0.5, 0.8]],
                    evidence=['F'], evidence_card=[2], state_names={"F":["low", "high"], "Q": ["bad", "good"]})
cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6, 0.3], [0.4, 0.7]],
                    evidence=['F'], evidence_card=[2], state_names={"F":["low", "high"], "D": ["weak", "strong"]})
cpd_w = TabularCPD(variable='W', variable_card=2, 
                    values=[[0.30, 0.5, 0.15, 0.25], [0.70, 0.5, 0.85, 0.75]],
                    evidence=['Q', 'D'], evidence_card=[2, 2], state_names={"Q":["bad", "good"], "D": ["weak", "strong"], "W": ["lose", "win"]})

# Add CPDs to model
model.add_cpds(cpd_f, cpd_q, cpd_d, cpd_w)

# Check Model
assert model.check_model()

In [8]:
_ = [print (cpd) for cpd in model.get_cpds()]

+---------+-----+
| F(low)  | 0.3 |
+---------+-----+
| F(high) | 0.7 |
+---------+-----+
+---------+--------+---------+
| F       | F(low) | F(high) |
+---------+--------+---------+
| Q(bad)  | 0.5    | 0.2     |
+---------+--------+---------+
| Q(good) | 0.5    | 0.8     |
+---------+--------+---------+
+-----------+--------+---------+
| F         | F(low) | F(high) |
+-----------+--------+---------+
| D(weak)   | 0.6    | 0.3     |
+-----------+--------+---------+
| D(strong) | 0.4    | 0.7     |
+-----------+--------+---------+
+---------+---------+-----------+---------+-----------+
| Q       | Q(bad)  | Q(bad)    | Q(good) | Q(good)   |
+---------+---------+-----------+---------+-----------+
| D       | D(weak) | D(strong) | D(weak) | D(strong) |
+---------+---------+-----------+---------+-----------+
| W(lose) | 0.3     | 0.5       | 0.15    | 0.25      |
+---------+---------+-----------+---------+-----------+
| W(win)  | 0.7     | 0.5       | 0.85    | 0.75      |
+---------+---

----
Calculate $P(W|F=\text{high})$

$$
\begin{align}
P(W|F=\text{high}) & = \\
& \propto P(W,F=\text{high}) \\
& = \sum_{q\in Q,d \in D} P(F=\text{high}, Q, D, W) \\
& = \sum_{q,d} P(F=\text{high})P(q|F=\text{high})P(d|F=\text{high})P(W|q,d) 
\end{align}
$$

In [16]:
P_w = None

# Calcuate the probabiity of winning and losing 
# and put it in a the P_w variable

# YOUR CODE HERE
# raise NotImplementedError()

p_win = \
0.7*0.2*0.3*0.7+\
0.7*0.2*0.7*0.5+\
0.7*0.8*0.3*0.85+\
0.7*0.8*0.7*0.75
#p(high)*p(bad,high)*p(weak,high)*p(bad,weak,win)

p_lose = \
0.7*0.2*0.3*0.3+\
0.7*0.2*0.7*0.5+\
0.7*0.8*0.3*0.15+\
0.7*0.8*0.7*0.25

unnorm = [p_win,p_lose]

print(f"p_win: {p_win}")
print(f"p_lose: {p_lose}")

P_w = [p_win/(p_win+p_lose)]

P_w

p_win: 0.5152
p_lose: 0.18479999999999996


[0.736]

In [ ]:
# This cell intentionaly left empty


----
Then we can use Variable Elimination to do the same inference. 

Variable Elimination is based on the following insight:

$$
\begin{align}
& \sum_{q,d} P(F=\text{high})P(q|F=\text{high})P(d|F=\text{high})P(W|q,d) \\
& = P(F=\text{high}) \sum_{q,d} P(q|F=\text{high})P(d|F=\text{high})P(W|q,d) \\
& = P(F=\text{high}) \sum_{q} P(q|F=\text{high})\sum_{d}P(d|F=\text{high})P(W|q,d) \\
\end{align}
$$

----

Now use the `VariabeElimination` functionalityin `pgmpy` to calcuate the same probability.


In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

from pgmpy.inference import VariableElimination
inference = VariableElimination(model)

P_w= inference.query(variables=['W'], evidence={'F': 'high'})

P_w

----
Here's a more complex example, using the indurance BN:

In [19]:
from pgmpy.utils import get_example_model

model = get_example_model('insurance')
model.get_cardinality()

defaultdict(int,
            {'Accident': np.int64(4),
             'Age': np.int64(3),
             'Airbag': np.int64(2),
             'AntiTheft': np.int64(2),
             'Antilock': np.int64(2),
             'CarValue': np.int64(5),
             'Cushioning': np.int64(4),
             'DrivHist': np.int64(3),
             'DrivQuality': np.int64(3),
             'DrivingSkill': np.int64(3),
             'GoodStudent': np.int64(2),
             'HomeBase': np.int64(4),
             'ILiCost': np.int64(4),
             'MakeModel': np.int64(5),
             'MedCost': np.int64(4),
             'Mileage': np.int64(4),
             'OtherCar': np.int64(2),
             'OtherCarCost': np.int64(4),
             'PropCost': np.int64(4),
             'RiskAversion': np.int64(4),
             'RuggedAuto': np.int64(3),
             'SeniorTrain': np.int64(2),
             'SocioEcon': np.int64(4),
             'Theft': np.int64(2),
             'ThisCarCost': np.int64(4),
             'T

In [17]:
print(model.get_cpds(node="Age"))

+-----------------+-----+
| Age(Adolescent) | 0.2 |
+-----------------+-----+
| Age(Adult)      | 0.6 |
+-----------------+-----+
| Age(Senior)     | 0.2 |
+-----------------+-----+


In [18]:
print(model.get_cpds(node="DrivQuality"))

+------------------------+-----+------------------------+
| DrivingSkill           | ... | DrivingSkill(Expert)   |
+------------------------+-----+------------------------+
| RiskAversion           | ... | RiskAversion(Cautious) |
+------------------------+-----+------------------------+
| DrivQuality(Poor)      | ... | 0.0                    |
+------------------------+-----+------------------------+
| DrivQuality(Normal)    | ... | 0.0                    |
+------------------------+-----+------------------------+
| DrivQuality(Excellent) | ... | 1.0                    |
+------------------------+-----+------------------------+


Can you calculate the probability of `DrivQuality` given `Age` for both `Adolescent` and `Senior` values of `Age`? 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()